# Interactive Examples on Project Aria Tools

### Notebook stuck?
Note that because of Jupyter issues, sometimes the code may stuck at visualization. We recommend **restart the kernels** and try again to see if the issue is resolved.

In [ ]:
# Specifics for Google Colab
google_colab_env = 'google.colab' in str(get_ipython())
if google_colab_env:
    print("Running from Google Colab, installing projectaria_tools and getting sample data")
    !pip install projectaria-tools
    !curl -O -J -L "https://github.com/facebookresearch/projectaria_tools/raw/main/data/mps_sample/sample.vrs"
    vrsfile = "sample.vrs"
else:
    print("Using a pre-existing projectaria_tool github repository")
    vrsfile = "../../data/mps_sample/sample.vrs"

In [ ]:
from projectaria_tools.core import data_provider, calibration
from projectaria_tools.core.image import InterpolationMethod
from projectaria_tools.core.sensor_data import TimeDomain, TimeQueryOptions
from projectaria_tools.core.stream_id import RecordableTypeId, StreamId
import numpy as np
from matplotlib import pyplot as plt

## Create data provider

In [ ]:
print(f"Creating data provider from {vrsfile}")
provider = data_provider.create_vrs_data_provider(vrsfile)
if not provider:
    print("Invalid vrs data provider")

# Retrieving image data

Goals:
- Learn how to retrieve Image data for a given Image stream

Key learnings:
- VRS contains data streams are identified with a Unique Identifier: stream_id
- Learn what are the Stream Ids used by Aria data (Slam, Rgb, EyeTracking)
- Learn that image data can be retrieved by using a record Index or a timestamp
- For each stream_id, index ranges from [0, get_num_data(stream_id)], and the same index for different streams could have different timestamps
- Query data from different sensors of the same timestamp can be done through `get_image_data_by_time_ns`, `get_imu_data_by_time_ns`, etc

In [ ]:
stream_mappings = {
    "camera-slam-left": StreamId("1201-1"),
    "camera-slam-right":StreamId("1201-2"),
    "camera-rgb":StreamId("214-1"),
    "camera-eyetracking":StreamId("211-1"),
}

axes = []
fig, axes = plt.subplots(1, 4, figsize=(12, 5))
fig.suptitle('Retrieving image data using Record Index')

# Query data with index
frame_index = 1
for idx, [stream_name, stream_id] in enumerate(list(stream_mappings.items())):
    image = provider.get_image_data_by_index(stream_id, frame_index)
    axes[idx].imshow(image[0].to_numpy_array(), cmap="gray", vmin=0, vmax=255)
    axes[idx].title.set_text(stream_name)
    axes[idx].tick_params(left=False, right=False, labelleft=False, labelbottom=False, bottom=False)
plt.show()

# Same example using Time
rgb_stream_id = StreamId('214-1')
plt.figure()
fig, axes = plt.subplots(1, 4, figsize=(12, 5))
fig.suptitle('Retrieving image data using Time')

time_domain = TimeDomain.DEVICE_TIME  # query data based on host time
option = TimeQueryOptions.CLOSEST # get data whose time [in TimeDomain] is CLOSEST to query time
start_time = provider.get_first_time_ns(rgb_stream_id, time_domain)

for idx, [stream_name, stream_id] in enumerate(list(stream_mappings.items())):
    image = provider.get_image_data_by_time_ns(stream_id, start_time, time_domain, option)
    axes[idx].imshow(image[0].to_numpy_array(), cmap="gray", vmin=0, vmax=255)
    axes[idx].title.set_text(stream_name)
    axes[idx].tick_params(left=False, right=False, labelleft=False, labelbottom=False, bottom=False)
plt.show()


# Summarize a VRS using thumbnails

Goals:
- Summarize a VRS using 10 image side by side

Key learnings:
- Image streams are identified with a Unique Identifier: stream_id
- PIL images can be created from Numpy array

In [ ]:
from PIL import Image
from tqdm import tqdm

rgb_stream_id = StreamId("214-1")

# Retrieve Start and End time for the given Sensor Stream Id
start_time = provider.get_first_time_ns(rgb_stream_id, time_domain)
end_time = provider.get_last_time_ns(rgb_stream_id, time_domain)

# Retrieve image size for the RGB stream
time_domain = TimeDomain.DEVICE_TIME  # query data based on host time
option = TimeQueryOptions.CLOSEST # get data whose time [in TimeDomain] is CLOSEST to query time

image_config = provider.get_image_configuration(rgb_stream_id)
width = image_config.image_width
height = image_config.image_height

sample_count = 10
resize_ratio = 10
thumbnail = newImage = Image.new(
    "RGB", (int(width * sample_count / resize_ratio), int(height / resize_ratio))
)
current_width = 0


# Samples 10 timestamps
sample_timestamps = np.linspace(start_time, end_time, sample_count)
for sample in tqdm(sample_timestamps):
    image_tuple = provider.get_image_data_by_time_ns(rgb_stream_id, int(sample), time_domain, option)
    image_array = image_tuple[0].to_numpy_array()
    image = Image.fromarray(image_array)
    new_size = (
        int(image.size[0] / resize_ratio),
        int(image.size[1] / resize_ratio),
    )
    image = image.resize(new_size).rotate(-90)
    thumbnail.paste(image, (current_width, 0))
    current_width = int(current_width + width / resize_ratio)

from IPython.display import Image 
display(thumbnail)

# Obtain mapping between stream_id and sensor label
Goals:
- In a vrs file, each sensor data is identitied through stream_id
- Learn mapping between stream_id and label for each sensor

Key learnings:
- VRS is using Unique Identifier for each stream called stream_id. 
- For each sensor data, it is attached with a stream_id, which contains two parts [RecordableTypeId, InstanceId]. 
- To get the actual readable name of each sensor,
we can use `get_label_from_stream_id` vise versa `get_stream_id_from_label`

In [ ]:
streams = provider.get_all_streams()
for stream_id in streams:
    label = provider.get_label_from_stream_id(stream_id)
    print(
        f"stream_id: [{stream_id}] convert to label: [{label}] and back: [{provider.get_stream_id_from_label(label)}]"
    )

# Get sensor data in a sequence based on data capture time
Goal:
- Obtain sensor data sequentially based on timestamp

Key learnings
- Default option activates all sensors and playback the entire dataset from vrs
- Setup option to only activate certain streams, truncate start/end time, and sample rate
- Obtain data from different sensor types
- `TimeDomain` are separated into four categories: `RECORD_TIME`, `DEVICE_TIME`, `HOST_TIME`, `TIME_CODE`

### Step 1: obtain default options that provides the whole dataset from VRS
* activates all sensor streams
* No truncation for first/last timestamp
* Subsample rate = 1 (do not skip any data per sensor)

In [ ]:
options = (
    provider.get_default_deliver_queued_options()
)  # default options activates all streams

### Step 2: set prefered deliver options
* truncate first/last time: `set_truncate_first_device_time_ns/set_truncate_last_device_time_ns()`
* subselect sensor streams to play: `activate_stream(stream_id)`
* skip sensor data : `set_subsample_rate(stream_id, rate)`

In [ ]:
options.set_truncate_first_device_time_ns(int(1e8))  # 0.1 secs after vrs first timestamp
options.set_truncate_last_device_time_ns(int(1e9))  # 1 sec before vrs last timestamp

# deactivate all sensors
options.deactivate_stream_all()
# activate only a subset of sensors
slam_stream_ids = options.get_stream_ids(RecordableTypeId.SLAM_CAMERA_DATA)
imu_stream_ids = options.get_stream_ids(RecordableTypeId.SLAM_IMU_DATA)

for stream_id in slam_stream_ids:
    options.activate_stream(stream_id)  # activate slam cameras
    options.set_subsample_rate(stream_id, 1)  # sample every data for each slam camera
    
for stream_id in imu_stream_ids:
    options.activate_stream(stream_id)  # activate imus
    options.set_subsample_rate(stream_id, 10)  # sample every 10th data for each imu

### Step 3: create iterator to deliver data
`TimeDomain` contains the following
* `RECORD_TIME`: timestamp stored in vrs index, fast to access, but not guaranteed which time domain
* `DEVICE_TIME`: capture time in device's timedomain, accurate
* `HOST_TIME`: arrival time in host computer's timedomain, may not be accurate
* `TIME_CODE`: capture in TimeSync server's timedomain


In [ ]:
iterator = provider.deliver_queued_sensor_data(options)
for sensor_data in iterator:
    label = provider.get_label_from_stream_id(sensor_data.stream_id())
    sensor_type = sensor_data.sensor_data_type()
    device_timestamp = sensor_data.get_time_ns(TimeDomain.DEVICE_TIME)
    host_timestamp = sensor_data.get_time_ns(TimeDomain.HOST_TIME)
    timecode_timestamp = sensor_data.get_time_ns(TimeDomain.TIME_CODE)
    print(
        f"""obtain data from {label} of type {sensor_type} with \n
        DEVICE_TIME: {device_timestamp} nanoseconds \n
        HOST_TIME: {host_timestamp} nanoseconds \n
        """
    )

# Random access data
Goal
- Access data from a stream randomly using a data index or a timestamp

Key learnings
- Sensor data can be obtained through index within the range of [0, number of data for this stream_id)

  - `get_sensor_data_by_index(stream_id, index)`
  - `get_image_data_by_index(stream_id, index)`
  - Access other sensor data by index interface is available in core/python/VrsDataProviderPyBind.h
  
- `TimeQueryOptions` has three options: `TimeQueryOptions.BEFORE`, `TimeQueryOptions.AFTER`, `TimeQueryOptions.CLOSEST`
- Query through index will provide the exact data vs query through a timestamp that is not exact, data nearby will be omitted base on `TimeQueryOptions`

In [ ]:
sensor_name = "camera-slam-right"
sensor_stream_id = provider.get_stream_id_from_label(sensor_name)

# get all image data by index
num_data = provider.get_num_data(sensor_stream_id)

for index in range(0, num_data):
    image_data = provider.get_image_data_by_index(sensor_stream_id, index)
    print(
        f"Get image: {index} with timestamp {image_data[1].capture_timestamp_ns}"
    )

### Sensor data can be obtained by timestamp (nanoseconds)
* Get stream time range `get_first_time_ns` and `get_last_time_ns`
* Specify timedomain: `TimeDomain.DEVICE_TIME` (default)
* Query data by queryTime
  * `TimeQueryOptions.BEFORE` (default): sensor_dataTime <= queryTime
  * `TimeQueryOptions.AFTER` : sensor_dataTime >= queryTime
  * `TimeQueryOptions.CLOSEST` : sensor_dataTime closest to queryTime

In [ ]:
time_domain = TimeDomain.DEVICE_TIME  # query data based on DEVICE_TIME
option = TimeQueryOptions.CLOSEST # get data whose time [in TimeDomain] is CLOSEST to query time

start_time = provider.get_first_time_ns(sensor_stream_id, time_domain)
end_time = provider.get_last_time_ns(sensor_stream_id, time_domain)

for time in range(start_time, end_time, int(1e7)):
    image_data = provider.get_image_data_by_time_ns(
        sensor_stream_id, time, time_domain, option
    )
    print(
        f"query time {time} and get capture image time {image_data[1].capture_timestamp_ns} within range {start_time} {end_time}"
    )

### Get sensor data configuration

In [ ]:
def image_config_example(config):
    print(f"device_type {config.device_type}")
    print(f"device_version {config.device_version}")
    print(f"device_serial {config.device_serial}")
    print(f"sensor_serial {config.sensor_serial}")
    print(f"nominal_rate_hz {config.nominal_rate_hz}")
    print(f"image_width {config.image_width}")
    print(f"image_height {config.image_height}")
    print(f"pixel_format {config.pixel_format}")

In [ ]:
config = provider.get_image_configuration(sensor_stream_id)
image_config_example(config)

# Calibration examples
Goal:
- Obtain camera extrinsics and intrinsics
- Learn to project a 3D point to camera frame

Key learnings
- Get calibration for different sensors using sensor labels
- Learn how to use extrinsics/intrinsics to project a 3D points to a given camera
- Reference frame convention

In [ ]:
device_calib = provider.get_device_calibration()
all_sensor_labels = device_calib.get_all_labels()
print(f"device calibration contains calibrations for the following sensors \n {all_sensor_labels}")

### Project a 3D point to camera frame

In this section we will learn how to retrieve calibration data and how to use it.
Aria calibration is defined by two objects: one defining the intrinsics (`rgb_calib.project` and `rgb_calib.unproject`) and one defining the extrinsics as a SE3 pose (`device_calib.get_transform_device_sensor(sensor_label`).

Intrinsics can be used to project a 3d point to the image plane or un-project a 2d point as a bearing vector. Extrinsics are used to set the camera in world coordinates at a given rotation and position in space.

### Reference frame convention

> `transform_sensor1_sensor3` = `transform_sensor1_sensor2` * `transform_sensor2_sensor3` \
> `point_in_sensor`: 3D point measured from sensor's reference frame \
> `point_in_sensor` = `transform_sensor1_sensor` * `point_in_sensor`

Device Frame: `device_calib.get_origin_label() = camera-slam-left`\
Sensor extrinsics: `device_calib.get_transform_device_sensor(sensor_label)`

In [ ]:
camera_name = "camera-rgb"
transform_device_camera = device_calib.get_transform_device_sensor(camera_name).to_matrix()
transform_camera_device = np.linalg.inv(transform_device_camera)
print(f"Device calibration origin label {device_calib.get_origin_label()}")
print(f"{camera_name} has extrinsics of \n {transform_device_camera}")

rgb_calib = device_calib.get_camera_calib("camera-rgb")
if rgb_calib is not None:
    # project a 3D point in device frame [camera-slam-left] to rgb camera
    point_in_device = np.array([0, 0, 10])
    point_in_camera = (
        np.matmul(transform_camera_device[0:3,0:3], point_in_device.transpose())
        + transform_camera_device[0:3,3]
    )

    maybe_pixel = rgb_calib.project(point_in_camera)
    if maybe_pixel is not None:
        print(
            f"Get pixel {maybe_pixel} within image of size {rgb_calib.get_image_size()}"
        )

### Get calibration data for other sensors
Aria is a multimodal capture device, each sensors calibration can be retrieved using the same interface. Only EyeTracking (`get_aria_et_camera_calib()`) and Audio calibration (`get_aria_microphone_calib()`) is a bit different since we have multiple sensors that share the same stream_id.

In [ ]:
et_calib = device_calib.get_aria_et_camera_calib()
if et_calib is not None:
    print(f"Camera {et_calib[0].get_label()} has image size {et_calib[0].get_image_size()}")
    print(f"Camera {et_calib[1].get_label()} has image size {et_calib[1].get_image_size()}")

imu_calib = device_calib.get_imu_calib("imu-left")
if imu_calib is not None:
    print(f"{imu_calib.get_label()} has extrinsics transform_Device_Imu:\n {imu_calib.get_transform_device_imu().to_matrix3x4()}")

### Undistort an image
You can remove distortions in an image in three steps. 

First, use the provider to access the image and the camera calibration of the stream. Then create a "linear" spherical camera model with `get_spherical_camera_calibration`. The function allows you to specify the image size as well as focal length of the model, assuming principal point is at the image center. Finally, apply `distort_by_calibration` function to distort the image.

In [ ]:
# input: retrieve image as a numpy array
sensor_name = "camera-rgb"
sensor_stream_id = provider.get_stream_id_from_label(sensor_name)
image_data = provider.get_image_data_by_index(sensor_stream_id, 0)
image_array = image_data[0].to_numpy_array()
# input: retrieve image distortion
device_calib = provider.get_device_calibration()
src_calib = device_calib.get_camera_calib(sensor_name)

# create output calibration: a linear model of image size 512x512 and focal length 150
# Invisible pixels are shown as black.
dst_calib = calibration.get_linear_camera_calibration(512, 512, 150, camera_name)

# distort image
rectified_array = calibration.distort_by_calibration(image_array, dst_calib, src_calib, InterpolationMethod.BILINEAR)

# visualize input and results
plt.figure()
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
fig.suptitle(f"Image undistortion (focal length = {dst_calib.get_focal_lengths()})")

axes[0].imshow(image_array, cmap="gray", vmin=0, vmax=255)
axes[0].title.set_text(f"sensor image ({sensor_name})")
axes[0].tick_params(left=False, right=False, labelleft=False, labelbottom=False, bottom=False)
axes[1].imshow(rectified_array, cmap="gray", vmin=0, vmax=255)
axes[1].title.set_text(f"undistorted image ({sensor_name})")
axes[1].tick_params(left=False, right=False, labelleft=False, labelbottom=False, bottom=False)
plt.show()

Note the rectified image shows a circular area of visible pixels. If you want the entire rectified image to be covered by pixels, you can increase the magnification.